In [8]:
import pandas as pd
import sqlite3
import sys
import os
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from data.snowflake_pull import get_snowflake_config, setconnection, run_query_to_df
import data.sql_lite_store as sql_lite_store
import data.snowflake_pull as snowflake_pull

import data.demand_pull as demand_pull
import agents.workingFlow as workingFlow

In [25]:
def find_project_root(start=None, markers=("pyproject.toml", "requirements.txt", ".git", ".env")):
    p = Path(start or os.getcwd()).resolve()
    for parent in (p, *p.parents):
        if any((parent / m).exists() for m in markers):
            return parent
    return Path.cwd().resolve()

ROOT = find_project_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

LOCAL_DB_PATH = ROOT / "data" / "inventory_data.db"
LOCAL_DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [275]:
df_demand = sql_lite_store.load_table("demand_data")
df_kepplerSplits = sql_lite_store.load_table("Keppler_Split_Perc")
df_vendor_cbm_old = sql_lite_store.load_table("Vendor_CBM")
df_CBM_Max = sql_lite_store.load_table("CBM_Max")


In [279]:
import importlib, data.snowflake_pull
importlib.reload(data.snowflake_pull)
config = get_snowflake_config()
conn = setconnection(config)
df_vendor_cbm = snowflake_pull.run_query_to_df(conn, snowflake_pull.SQL_Vendor_CBM)

In [280]:
df_vendor_cbm_old.columns

Index(['PRODUCT', 'CHW_SKU_NUMBER', 'MC1_NAME', 'MC2_NAME', 'MC3_NAME',
       'BRAND', 'CUSTOMER_EARLIEST_TARGET_DATE', 'EARLIEST_TARGET_DATE',
       'CHW_MOQ_LEVEL', 'CHW_OTB', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'CHW_MASTER_CASE_PACK',
       'CHW_MASTER_CARTON_CBM'],
      dtype='object')

In [281]:
df_vendor_cbm.columns

Index(['PRODUCT', 'CHW_SKU_NUMBER', 'MC1_NAME', 'MC2_NAME', 'MC3_NAME',
       'BRAND', 'CUSTOMER_EARLIEST_TARGET_DATE', 'EARLIEST_TARGET_DATE',
       'CHW_MOQ_LEVEL', 'CHW_OTB', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'PRIMARY_SUPPLIER',
       'CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM'],
      dtype='object')

In [282]:

config = get_snowflake_config()
conn = setconnection(config)

df_skuSupplySnapshot = snowflake_pull.run_query_to_df(conn, snowflake_pull.SQL_SKU_Supply_Snapshot)

In [283]:
keep_cols_1 = [
    'MC1_NAME', 'MC2_NAME', 'MC3_NAME', 'BRAND',
    "CHW_SKU_NUMBER",
    "CHW_MOQ_LEVEL",
    "CHW_PRIMARY_SUPPLIER_NAME",
    "CHW_PRIMARY_SUPPLIER_NUMBER",
    'CHW_OTB',
    "CHW_MASTER_CASE_PACK",
    "CHW_MASTER_CARTON_CBM",]
    
df_vendor_cbm = df_vendor_cbm[keep_cols_1]



In [284]:

df_demand = df_demand[["product_part_number", "Final Buy Qty"]].copy()
df_demand = df_demand.rename(columns={'product_part_number': 'CHW_SKU_NUMBER', 'Final Buy Qty': 'Planned_Demand'})
df_demand['CHW_SKU_NUMBER'] = df_demand['CHW_SKU_NUMBER'].astype(str).str.strip()


In [244]:
df_demand.columns

Index(['CHW_SKU_NUMBER', 'Planned_Demand'], dtype='object')

In [285]:

df_demand = df_demand.merge(
    df_vendor_cbm,
    how='left',
    on=['CHW_SKU_NUMBER'])

In [286]:
def keep_first_max_avglt(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["AVG_LT"] = pd.to_numeric(df["AVG_LT"], errors="coerce")
    idx = df.groupby("SKU")["AVG_LT"].idxmax()   # index of max AVG_LT per SKU
    return df.loc[idx].reset_index(drop=True)

In [255]:
df_skuSupplySnapshot.columns

Index(['MC1', 'MC2', 'BRAND', 'PSKU', 'SKU', 'PRODUCT_NAME', 'PUBBED', 'OH',
       'T90_DAILY_AVG', 'F90_DAILY_AVG', 'AVG_LT', 'OO', 'NEXT_DELIVERY',
       'T90_DOS_OH', 'F90_DOS_OH', 'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW',
       'ALERT'],
      dtype='object')

In [287]:
keep_cols_2 = [
    'SKU','PRODUCT_NAME', 'OH',
    'T90_DAILY_AVG', 'F90_DAILY_AVG', 'AVG_LT', 'OO', 'NEXT_DELIVERY',
    'T90_DOS_OH', 'F90_DOS_OH', 'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW',
    'ALERT',]
    
df_skuSupplySnapshot = df_skuSupplySnapshot[keep_cols_2]

In [288]:
#combine the 2 tables,
df_demand['CHW_SKU_NUMBER'] = df_demand['CHW_SKU_NUMBER'].astype(str).str.strip()
df_skuSupplySnapshot['SKU']= df_skuSupplySnapshot['SKU'].astype(str).str.strip()
#detected duplicated SKU in df_skuSupplySnapshot, keep the first max AVG_LT
df_skuSupplySnapshot = keep_first_max_avglt(df_skuSupplySnapshot)


df_sku_data = df_demand.merge(
    df_skuSupplySnapshot,
    how='left',
    left_on='CHW_SKU_NUMBER',
    right_on='SKU'
)


In [289]:
df_sku_data.columns

Index(['CHW_SKU_NUMBER', 'Planned_Demand', 'MC1_NAME', 'MC2_NAME', 'MC3_NAME',
       'BRAND', 'CHW_MOQ_LEVEL', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'CHW_OTB', 'CHW_MASTER_CASE_PACK',
       'CHW_MASTER_CARTON_CBM', 'SKU', 'PRODUCT_NAME', 'OH', 'T90_DAILY_AVG',
       'F90_DAILY_AVG', 'AVG_LT', 'OO', 'NEXT_DELIVERY', 'T90_DOS_OH',
       'F90_DOS_OH', 'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW', 'ALERT'],
      dtype='object')

In [290]:

# Drop requested columns (only if present)
avg_lt_mean = df_sku_data['AVG_LT'].mean(skipna=True)
num_cols = ["AVG_LT", "F90_DAILY_AVG", "OH", "OO", "Planned_Demand", "CHW_MOQ_LEVEL"]
df_sku_data[num_cols] = df_sku_data[num_cols].apply(pd.to_numeric, errors="coerce").astype(float)
df_sku_data['AVG_LT'] = df_sku_data['AVG_LT'].fillna(avg_lt_mean)
df_sku_data['OO'] = df_sku_data['OO'].fillna(0)
df_sku_data['CHW_MOQ_LEVEL'] = df_sku_data['CHW_MOQ_LEVEL'].fillna(0)
df_sku_data['OH'] = df_sku_data['OH'].fillna(0)

In [266]:
df_sku_data.dtypes

CHW_SKU_NUMBER                  object
Planned_Demand                 float64
MC1_NAME                        object
MC2_NAME                        object
MC3_NAME                        object
BRAND                           object
CHW_MOQ_LEVEL                  float64
CHW_PRIMARY_SUPPLIER_NUMBER     object
CHW_OTB                          int64
CHW_MASTER_CASE_PACK             int64
CHW_MASTER_CARTON_CBM           object
SKU                             object
PRODUCT_NAME                    object
OH                             float64
T90_DAILY_AVG                   object
F90_DAILY_AVG                  float64
AVG_LT                         float64
OO                             float64
NEXT_DELIVERY                   object
T90_DOS_OH                     float64
F90_DOS_OH                     float64
F90_DOS_OO                     float64
T90_BELOW                       object
F90_BELOW                       object
ALERT                           object
baseConsumption          

In [292]:
df_sku_data["baseConsumption"] = np.where(
    df_sku_data["F90_DAILY_AVG"].notna(),
    (df_sku_data["AVG_LT"] + 4 * 7) * df_sku_data["F90_DAILY_AVG"],
    0.0
)

df_sku_data["bufferConsumption"] = np.where(
    df_sku_data["F90_DAILY_AVG"].notna(),
    (df_sku_data["AVG_LT"] + 8 * 7) * df_sku_data["F90_DAILY_AVG"],
    df_sku_data["Planned_Demand"]
)

base_qty = df_sku_data["baseConsumption"] - (df_sku_data["OH"].fillna(0) + df_sku_data["OO"].fillna(0))
buffer_qty = df_sku_data["bufferConsumption"] - (df_sku_data["OH"].fillna(0) + df_sku_data["OO"].fillna(0))

df_sku_data["baseDemand"] = np.maximum(base_qty, 0)
df_sku_data["bufferDemand"] = np.maximum(buffer_qty, 0)
df_sku_data["baseDemand"] = np.minimum(df_sku_data["baseDemand"], df_sku_data["Planned_Demand"].fillna(0))
df_sku_data["excess_demand"] = np.maximum(df_sku_data["Planned_Demand"].fillna(0) - df_sku_data["bufferDemand"], 0)

df_sku_data["baseDemand"] = np.where(
    df_sku_data["baseConsumption"] == 0,
    df_sku_data["Planned_Demand"],             # if baseConsumption == 0 → use planned_demand
    df_sku_data["baseDemand"]                  # otherwise keep existing baseDemand
)

#snapping baseDemand to MOQ, since its not really a choice
df_sku_data["baseDemand"] = np.maximum(df_sku_data["baseDemand"], df_sku_data["CHW_MOQ_LEVEL"])

#snapping baseDemand to the higher of mcp multiples
m = df_sku_data["CHW_MASTER_CASE_PACK"]
df_sku_data["baseDemand"] = np.ceil(df_sku_data["baseDemand"] / m) * m
df_sku_data["excess_demand"] = np.floor(df_sku_data["excess_demand"] / m) * m




In [293]:
df_sku_data

,CHW_SKU_NUMBER,Planned_Demand,MC1_NAME,MC2_NAME,MC3_NAME,BRAND,CHW_MOQ_LEVEL,CHW_PRIMARY_SUPPLIER_NAME,CHW_PRIMARY_SUPPLIER_NUMBER,CHW_OTB,...,F90_DOS_OH,F90_DOS_OO,T90_BELOW,F90_BELOW,ALERT,baseConsumption,bufferConsumption,baseDemand,bufferDemand,excess_demand
0,139518,1000.0,Hard Goods,Apparel,Apparel,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,NaN,NaN,NaN,NaN,NaN,0.00,1000.00,1000.0,1000.00,0.0
1,139519,3108.0,Hard Goods,Apparel,Apparel,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,NaN,NaN,NaN,NaN,NaN,0.00,3108.00,3125.0,3108.00,0.0
2,162803,5806.0,Hard Goods,Apparel,Apparel,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,79.21,64.79,False,True,False,15430.25,18217.65,1200.0,3882.65,1800.0
3,248629,1277.0,Hard Goods,Apparel,Apparel,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,42.75,251.68,False,True,False,2309.50,2726.70,0.0,0.00,1250.0
4,608214,1105.0,Hard Goods,Apparel,Apparel,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,124.36,40.08,False,True,False,4454.70,5259.42,0.0,533.42,432.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2085542,2500.0,Hard Goods,Toys,Plush Toys,Frisco,0.0,P20837-KIDS FIRST ASIA CO LTD,P20837,False,...,NaN,NaN,NaN,NaN,NaN,0.00,2500.00,2502.0,2500.00,0.0
284,2085606,1500.0,Hard Goods,Toys,Subscription Boxes & Variety Packs,Frisco,0.0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,False,...,NaN,NaN,NaN,NaN,NaN,0.00,1500.00,1512.0,1500.00,0.0
285,2029310,1200.0,Hard Goods,Toys,Plush Toys,Frisco,0.0,P20837-KIDS FIRST ASIA CO LTD,P20837,False,...,NaN,NaN,NaN,NaN,NaN,0.00,1200.00,1224.0,1200.00,0.0
286,2031726,1200.0,Hard Goods,Toys,Plush Toys,Frisco,0.0,P20837-KIDS FIRST ASIA CO LTD,P20837,False,...,NaN,NaN,NaN,NaN,NaN,0.00,1200.00,1200.0,1200.00,0.0


In [311]:
df_sku_data.columns

Index(['CHW_SKU_NUMBER', 'Planned_Demand', 'MC1_NAME', 'MC2_NAME', 'MC3_NAME',
       'BRAND', 'CHW_MOQ_LEVEL', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'CHW_OTB', 'CHW_MASTER_CASE_PACK',
       'CHW_MASTER_CARTON_CBM', 'SKU', 'PRODUCT_NAME', 'OH', 'T90_DAILY_AVG',
       'F90_DAILY_AVG', 'AVG_LT', 'OO', 'NEXT_DELIVERY', 'T90_DOS_OH',
       'F90_DOS_OH', 'F90_DOS_OO', 'T90_BELOW', 'F90_BELOW', 'ALERT',
       'baseConsumption', 'bufferConsumption', 'baseDemand', 'bufferDemand',
       'excess_demand'],
      dtype='object')

In [321]:
print(df_kepplerSplits.columns)
print(df_vendor_cbm.columns)
print(df_CBM_Max.columns)


Index(['ITEM_ID', 'CARTONIZATION_FLAG', 'TOTAL_STAT_FCAST', 'TLA1_FCAST',
       'TNY1_FCAST', 'MDT1_FCAST', 'TLA1_FRAC', 'TNY1_FRAC', 'MDT1_FRAC'],
      dtype='object')
Index(['MC1_NAME', 'MC2_NAME', 'MC3_NAME', 'BRAND', 'CHW_SKU_NUMBER',
       'CHW_MOQ_LEVEL', 'CHW_PRIMARY_SUPPLIER_NAME',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'CHW_OTB', 'CHW_MASTER_CASE_PACK',
       'CHW_MASTER_CARTON_CBM'],
      dtype='object')
Index(['vendor_number', 'vendor_name', 'CBM Max', 'Unnamed: 3',
       'upload_timestamp'],
      dtype='object')


In [384]:
import importlib, states.state_loader
import states.ChewySkuState
importlib.reload(states.state_loader)
importlib.reload(states.ChewySkuState)


<module 'states.ChewySkuState' from 'C:\\genAIProjects\\MBM_WTF\\states\\ChewySkuState.py'>

In [385]:
sku_data_state_list = states.state_loader.df_to_chewy_sku_states(df_sku_data)

In [388]:
import importlib, states.state_loader
importlib.reload(states.state_loader)
importlib.reload(states.vendorState)
importlib.reload(states.ChewySkuState)


<module 'states.ChewySkuState' from 'C:\\genAIProjects\\MBM_WTF\\states\\ChewySkuState.py'>

In [389]:

vendor_state_list = states.state_loader.df_to_vendor_states(df_sku_data, df_CBM_Max, sku_data_state_list)

In [187]:
print(vendor_state_list[3].Demand_skus[1])

parent_product_part_number='191898.0' product_part_number='165206' product_name='Frisco Quilted Orthopedic Pillow Lounger Cat & Dog Bed with Removable Cover, Beige, XX-Large' vendor_Code='B000064' vendor_name='PM&J LLC - DIRECT IMPORT' vendor_purchaser_code='JADESCHACK' MOQ=252 MCP=6 case_pk_CBM=0.48585 planned_demand=252.0 vendor_earliest_ETD='2026-01-01 00:00:00' MC1='Hard Goods' MC2='Beds & Furniture' BRAND='' SKU=None PUBBED='True' OH=215 T90_DAILY_AVG=0.97 F90_DAILY_AVG=2.76 AVG_LT=143 ost_ord=0 Next_Delivery=None T90_DOS_OH=221.65 F90_DOS_OH=77.9 F90_DOS_OO=0.0 T90_BELOW=0.0 F90_BELOW=1.0 baseConsumption=471.96 bufferConsumption=549.24 baseDemand=252.0 bufferDemand=334.24 excess_demand=0.0 demand_by_dest=None


In [181]:
v = next((v for v in vendor_state_list if str(v.vendor_Code) == "P000398"), None)
print(v)

vendor_Code='P000398' CBM_Max=68.0


In [390]:
df_kepplerSplits.columns

Index(['ITEM_ID', 'CARTONIZATION_FLAG', 'TOTAL_STAT_FCAST', 'TLA1_FCAST',
       'TNY1_FCAST', 'MDT1_FCAST', 'TLA1_FRAC', 'TNY1_FRAC', 'MDT1_FRAC'],
      dtype='object')

In [391]:
df_kepplerSplits1 = snowflake_pull.run_query_to_df(conn, snowflake_pull.SQL_KEPLER_SPLITS)

In [392]:
df_kepplerSplits1

,ITEM_ID,CARTONIZATION_FLAG,TOTAL_STAT_FCAST,TLA1_FCAST,TNY1_FCAST,MDT1_FCAST
0,1781062,true,0.062311,0.030142,0.032170,0.000000
1,153735,true,0.062311,0.030561,0.031750,0.000000
2,740102,true,67.967274,32.340153,35.627122,0.000000
3,1099174,true,0.001702,0.000833,0.000869,0.000000
4,169661,true,19.829842,10.076857,8.493933,1.259052
...,...,...,...,...,...,...
4929,144849,false,2.443985,1.249958,0.483682,0.710344
4930,220511,true,0.537346,0.182112,0.355235,0.000000
4931,278140,true,0.679159,0.097466,0.581693,0.000000
4932,1216750,true,0.000999,0.000476,0.000523,0.000000


In [11]:
import preprocessing.data_preprocessing as data_preprocessing


df_sku_data = data_preprocessing.process_demand_data()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZJdb9owFIb%2FSuRdJ3b4ENQCKgqriEQ3Viio3JnkEDwSO%2FNxGrJfPxNK1V600u4i53ntx37P4PaUZ94LGJRaDUkYMOKBinUiVTokT6t7v088tEIlItMKhqQGJLejAYo8K%2Fi4tAf1CH9KQOu5jRTy5seQlEZxLVAiVyIH5Dbmy%2FHDnLcCxgujrY51Rt5Fvk4IRDDWGV4jCUqnd7C24JRWVRVU7UCblLYYY5TdUEedkW9X%2FuTu9AkfUtY5845w%2BOLV7U6qyxN8pbW7QMhnq9XCX%2Fxcrog3vqpOtMIyB7ME8yJjeHqcXwTQGUxm3zfPQYk%2BCLR%2BGKDS1T4TR4h1XpTW7Rm4L7qHhGY6le7a0XRIiqNMlpv1Q7FJkrqX7vY3cSqj6Fd9Z9qz577qrI%2Fd7W43r7dRyU46Jt762mvr3GuEWEKkzm1at8RaXT8M%2FbC%2FCju82%2BPtTtDusi3xpq5NqYRtklfl%2BABVHeijFY2aKAr6Zk3hdMz%2FyjTs15tu37R%2FM3bqUURNz9WSy7Tw5ngz%2Br83GND32dex%2B%2BGaiKYLncm49u61yYX9vKgwCJsVmfj7BuWQC5mNk8QAoissy3Q1MSCsm25rSiB0dDn143yP%2FgE%3D&RelayState=ve

In [27]:
import importlib
import data.sql_lite_store as sql_lite_store
importlib.reload(sql_lite_store )
sql_lite_store.save_table(df_sku_data, "df_sku_data")

(True, 288)

In [30]:
df_sku_data = sql_lite_store.load_table("df_sku_data")
df_CBM_Max = sql_lite_store.load_table("CBM_Max")
df_kepplerSplits = sql_lite_store.load_table("Keppler_Split_Perc")

In [54]:
import importlib
import preprocessing.data_preprocessing as data_preprocessing
importlib.reload(data_preprocessing )

<module 'preprocessing.data_preprocessing' from 'C:\\genAIProjects\\MBM_WTF\\preprocessing\\data_preprocessing.py'>

In [55]:
demand_by_Dest = data_preprocessing.split_base_demand_by_dest(df_sku_data, df_kepplerSplits)

AttributeError: module 'preprocessing.data_preprocessing' has no attribute 'split_base_demand_by_dest'

In [56]:
intermediate_demand = sql_lite_store.load_table("intermediate_demand")

In [58]:
intermediate_demand.head(2)

,CHW_SKU_NUMBER,PRODUCT,MC1_NAME,MC2_NAME,MC3_NAME,BRAND,CUSTOMER_EARLIEST_TARGET_DATE,EARLIEST_TARGET_DATE,CHW_MOQ_LEVEL,CHW_OTB,CHW_PRIMARY_SUPPLIER_NAME,CHW_PRIMARY_SUPPLIER_NUMBER,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM,unallocated_units,DEST,Demand,cases_needed
0,139518,Frisco Birthday Cake Dog & Cat Hat,Hard Goods,Apparel,Apparel,Frisco,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,Colorway,0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,50,0.052896,0.0,MDT1,100,2
1,139519,Frisco Birthday Cake Dog & Cat Hat,Hard Goods,Apparel,Apparel,Frisco,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,Colorway,0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",3755,25,0.070756,8.0,MDT1,250,10
